In [107]:
# import zip_longest from itertools
from itertools import zip_longest
import pandas as pd


In [143]:
# Print the number of lines in the file from the shell script
no_of_lines = ! wc -l "MetObjects.csv"
no_of_lines

['12108 MetObjects.csv']

In [98]:
def return_dict_from_csv_line(header, line):
    # zip them
    zipped_line = zip_longest(header, line, fillvalue=None)
    # Use dict comprehension to generate the final dict
    ret_dict = {kv[0]: kv[1] for kv in zipped_line}
    return ret_dict

In [158]:
# Reading the first line into a dataframe

with open("MetObjects.csv", "r", encoding="latin1") as fd:
    first_line = fd.readline()
    header = first_line.replace("\n", "").split(",")
col_header = header

In [159]:
col_header

['"Object Number',
 'Is Highlight',
 'Is Public Domain',
 'Object ID',
 'Department',
 'Object Name',
 'Title',
 'Culture',
 'Period',
 'Dynasty',
 'Reign',
 'Portfolio',
 'Artist Role',
 'Artist Prefix',
 'Artist Display Name',
 'Artist Display Bio',
 'Artist Suffix',
 'Artist Alpha Sort',
 'Artist Nationality',
 'Artist Begin Date',
 'Artist End Date',
 'Object Date',
 'Object Begin Date',
 'Object End Date',
 'Medium',
 'Dimensions',
 'Credit Line',
 'Geography Type',
 'City',
 'State',
 'County',
 'Country',
 'Region',
 'Subregion',
 'Locale',
 'Locus',
 'Excavation',
 'River',
 'Classification',
 'Rights and Reproduction',
 'Link Resource',
 'Metadata Date',
 'Repository',
 'Tags"']

In [170]:
# opening the file to read its contents
# using the encoding as latin1
# returning the line numbers
i = 0
lines = []
with open("MetObjects.csv", "r", encoding="latin1") as fd:
    next(fd) # Skipping the first row
    for i, line in enumerate(fd):
        line = line.replace("\n", "").split(",")
        lines.append(line)
        i =+ i
print(i)

12107


In [204]:
metobj_df = pd.DataFrame(data = lines)
# print(metobj_df.shape, metobj_df.head())
metobj_df[41].unique()

array(['', 'Metal', 'Glass', ...,
       'http://www.metmuseum.org/art/collection/search/10921',
       'http://www.metmuseum.org/art/collection/search/10949',
       'http://www.metmuseum.org/art/collection/search/11793'],
      dtype=object)

In [172]:
metobj_df = pd.DataFrame(data = lines, columns=col_header)
print(metobj_df.shape, metobj_df.head())

ValueError: 44 columns passed, passed data had 64 columns

In [124]:
with open("MetObjects.csv", "r", encoding="latin1") as fd:
    for i, line in enumerate(fd):
        line = line.replace("\n", "").split(",")
        df_file = line
        if i > 2:
            break

df_file

'"67.265.9'

AttributeError: 'list' object has no attribute 'shape'